# **LCEL and Chains**

**LangChain Expression Language** allows to build compositions of chains by putting components together. It leverages the `|` pipe symbol to compose LangChain components.

**Chains** refer to sequences of calls - whether to an LLM, a tool, or a data preprocessing step. The primary supported way to do this is with LCEL.

**Chains** allows us to link the output of one LLM call as the input of another call.

In [1]:
# Setup API Key

f = open('keys/.openai_api_key.txt')

OPENAI_API_KEY = f.read()

In [2]:
from langchain_openai import ChatOpenAI

# Set the OpenAI Key and initialize a ChatModel
chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

/Users/kanavbansal/Developer/.env_jupyter/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a helpful AI bot who expertise in Data Science Tutor. 
                      You are known to make any complex topic simpler even for a beginner."""),
        ("human", "What is {topic}?")
    ]
)

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [5]:
chain = chat_template | chat_model | output_parser

### **chain.invoke()**

In [6]:
user_input = {"topic": "Feature Selection"}

response = chain.invoke(user_input)

print(response)

Feature selection is the process of selecting a subset of relevant features (variables, predictors) from the original set of features in a dataset. The goal of feature selection is to improve the performance of a machine learning model by reducing overfitting, increasing model interpretability, and reducing computation time.

There are three main types of feature selection techniques:
1. Filter methods: These methods select features based on their statistical properties, such as correlation with the target variable or variance. Common techniques include Pearson correlation coefficient and chi-square test.
2. Wrapper methods: These methods evaluate different subsets of features using a specific machine learning algorithm and select the subset that produces the best performance. Examples include forward selection, backward elimination, and recursive feature elimination.
3. Embedded methods: These methods perform feature selection as part of the model building process. Techniques such as 

### **chain.stream()**

Large language models can take several seconds to generate a complete response to a query. This is far slower than the **~200-300 ms threshold** at which an application feels responsive to an end user.

The key strategy to make the application feel more responsive is to show intermediate progress; viz., to stream the output from the model **token by token**.

In [7]:
for chunk in chain.stream(user_input):
    print(chunk, end="")

Feature selection is the process of selecting a subset of relevant features (variables, predictors) from the original set of features in a dataset. The goal of feature selection is to improve model performance by reducing overfitting, decreasing computational complexity, and increasing model interpretability.

There are various techniques for feature selection, including:

1. Filter methods: These methods select features based on statistical measures like correlation, chi-squared test, or mutual information.

2. Wrapper methods: These methods evaluate different subsets of features using a specific machine learning algorithm to determine the best subset.

3. Embedded methods: These methods perform feature selection as part of the model building process, where feature importance is determined during training.

By selecting the most relevant features, we can improve the efficiency and accuracy of machine learning models, leading to better predictions and insights from the data.

## **Case Study**

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [12]:
WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in requests about the topics and write organized research reports on those topics.
Also you share the appropriate references at the end of report."""

HUMAN_PROMPT_1 = """Write an organized research report about {topic}."""

writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_1)
])


writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

writer_chain = writer_chat_template | writer_chat_model | output_parser

research_report = writer_chain.invoke({"topic": "how transformers algorithm works?"})

print(research_report)

# Research Report: Understanding the Transformer Algorithm

## Abstract
The Transformer algorithm has revolutionized the field of natural language processing (NLP) and machine learning since its introduction in 2017. This report provides a comprehensive overview of the architecture, mechanisms, and applications of the Transformer model, highlighting its significance in various domains.

## 1. Introduction
The Transformer model, introduced by Vaswani et al. in the paper "Attention is All You Need," has become a foundational architecture for many state-of-the-art NLP tasks. Unlike previous models that relied heavily on recurrent neural networks (RNNs) and convolutional neural networks (CNNs), the Transformer utilizes a self-attention mechanism that allows for parallelization and improved performance on long-range dependencies in data.

## 2. Architecture of the Transformer
The Transformer architecture consists of an encoder-decoder structure, each composed of multiple layers. 

### 2.1 E

In [13]:
REVIEWER_SYS_PROMPT = """You are a reviewer for research reports. 
You take in research reports and provide a feedback on them."""

HUMAN_PROMPT_2 = """Provide feedback as 5 concise bullet points on this research report: 

{input}"""

reviewer_chat_template = ChatPromptTemplate.from_messages([
    ("system", REVIEWER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_2)
])

reviewer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, 
                                 model="gpt-4o-mini", 
                                 temperature=0.2)

reviewer_chain = reviewer_chat_template | reviewer_chat_model | output_parser

report_feedback = reviewer_chain.invoke({"input": research_report})

print(report_feedback)

- **Clarity and Structure**: The report is well-structured and clearly outlines the key components of the Transformer algorithm, making it accessible for readers with varying levels of expertise in NLP and machine learning.

- **Depth of Content**: While the report provides a solid overview of the architecture and mechanisms, it could benefit from deeper exploration of the implications of the self-attention mechanism and its advantages over RNNs and CNNs.

- **Examples and Applications**: The applications section is a strong point; however, including specific case studies or examples of successful implementations would enhance the practical understanding of the Transformer's impact across different domains.

- **References and Citations**: The references are relevant and up-to-date, but it would be beneficial to include more recent studies or advancements in Transformer models post-2021 to reflect ongoing developments in the field.

- **Future Directions**: The conclusion mentions the 

In [14]:
# Let's now write a final report having incorporated the feedback from the REVIEWER chain

FINAL_WRITER_SYS_PROMPT = """You are a research assistant and scientific writer.
You take in a research report in a set of bullet points with feedback to improve.
You revise the research report based on the feedback and write a final version."""

HUMAN_PROMPT_3 = """Write a reviewed and improved version of research report: 

{input_report}

based on this feedback:

{input_feedback}"""

final_writer_chat_template = ChatPromptTemplate.from_messages([
    ("system", FINAL_WRITER_SYS_PROMPT), 
    ("human", HUMAN_PROMPT_3)
])


final_writer_chat_model = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                               model="gpt-4o-mini",
                               temperature=0.0)

output_parser = StrOutputParser()

final_writer_chain = final_writer_chat_template | final_writer_chat_model | output_parser

final_report = final_writer_chain.invoke({"input_report": research_report, "input_feedback": report_feedback})

print(final_report)

# Research Report: Understanding the Transformer Algorithm

## Abstract
The Transformer algorithm has revolutionized the field of natural language processing (NLP) and machine learning since its introduction in 2017. This report provides a comprehensive overview of the architecture, mechanisms, and applications of the Transformer model, highlighting its significance in various domains. Additionally, it explores the implications of the self-attention mechanism, presents case studies of successful implementations, and discusses emerging trends and challenges in the field.

## 1. Introduction
The Transformer model, introduced by Vaswani et al. in the seminal paper "Attention is All You Need," has become a foundational architecture for many state-of-the-art NLP tasks. Unlike previous models that relied heavily on recurrent neural networks (RNNs) and convolutional neural networks (CNNs), the Transformer utilizes a self-attention mechanism that allows for parallelization and improved perform